# <span style="color: teal">Data Crawler</span>

### Instalando Pacotes

In [ ]:
%pip install pandas
%pip install pyyaml

### Obtendo Dados da Fonte

In [ ]:
import yaml
import pandas as pd
import urllib.parse

In [ ]:
# Parâmetros da requisição
parameters = {"dataTransacaoInicio": "01/01/2019", "dataTransacaoFim": "31/12/2019"} 
query_string = urllib.parse.urlencode(parameters)
query_string

In [ ]:
# Lendo as configurações
with open('.\configurations.yml') as file:
    configurations = yaml.load(file, Loader=yaml.FullLoader)
    print(configurations)

In [ ]:
df = pd.DataFrame()

def get_data_from_source():
    """Collect data from data source."""
    page = 0
    df = pd.DataFrame()
    while True:
        try:
            url = F"{configurations['DataSource']['Host']}{configurations['DataSource']['Route']}?{query_string}&pagina=" + str(page+1)
            api_response = pd.read_json(url)
            
            if page == 0:
                df = api_response # create new dataframe
            else:
                df = pd.concat([df, api_response]) # concat received data to created dataframe
                
            milestone(page)
            page += 1
        except:
             break
    return df

def milestone(page:int, mark=10):
    if page == 0:
        print("- Crawling Inciado -")
        return
        
    if page % mark == 0: 
        print('{} páginas lidas'.format(page))

In [ ]:
def data_transformation():
    df['valorTransacao'] = df['valorTransacao'].str.replace(' ','').str.replace('.','').str.replace(',','.').astype(float)
    df.reset_index(drop=True, inplace=True)
    #df[['id_cartao','codigo_cartao','descricao_cartao']] = df['tipoCartao'].apply(pd.Series)

In [ ]:
df = get_data_from_source()
df.head()